In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/My Drive/ar_reviews_100k.tsv.zip" -d "/content/drive/My Drive"

Archive:  /content/drive/My Drive/ar_reviews_100k.tsv.zip
  inflating: /content/drive/My Drive/ar_reviews_100k.tsv  


In [3]:
!pip install -U pip
!pip install "tensorflow_gpu>=2.0.0"
!pip install transformers==2.5.1
!pip install ktrain

     |████████████████████████████████| 1.5MB 4.5MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 320.4 MB 18 kB/s 
     |████████████████████████████████| 499 kB 5.5 MB/s 
     |████████████████████████████████| 1.1 MB 93.0 MB/s 
     |████████████████████████████████| 3.7 MB 82.5 MB/s 
     |████████████████████████████████| 883 kB 85.4 MB/s 
     |████████████████████████████████| 129 kB 77.0 MB/s 
     |████████████████████████████████| 6.7 MB 69.3 MB/s 
     |████████████████████████████████| 69 kB 8.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893259 sha256=2416bc38ebd7fcf7d654cfe62276d8b373e78f757437ba2589d9c9b854d57b59
  Stored in directory: /root/.cache/pip/wheels/49/25/98/cdea9c79b2d9a22ccc59540b1784b67f06b633378e97f58da2
Successfully built sacremoses
ERROR: After October 2020 you may experience errors when installing or upd

In [4]:
# forked versions of eli5 and stellargraph (required by ktrain)
!pip install git+https://github.com/amaiya/eli5@tfkeras_0_10_1 
!pip install git+https://github.com/amaiya/stellargraph@no_tf_dep_082

  Cloning https://github.com/amaiya/eli5 (to revision tfkeras_0_10_1) to /tmp/pip-req-build-jf71a_3f
  Created wheel for eli5: filename=eli5-0.10.1-py2.py3-none-any.whl size=106831 sha256=d9a4b6ce7d37677fdf4112b98d6dc31522623a7024ab5df998dc715dd2d3fe13
  Stored in directory: /tmp/pip-ephem-wheel-cache-zqyihtgu/wheels/93/23/c2/479f99e6e981887ac70af72d4ff763471acf7184d1b80a9268
Successfully built eli5
  Cloning https://github.com/amaiya/stellargraph (to revision no_tf_dep_082) to /tmp/pip-req-build-u1zoklfo
     |████████████████████████████████| 1.7 MB 4.7 MB/s 
  Created wheel for stellargraph: filename=stellargraph-0.8.2-py3-none-any.whl size=146374 sha256=d129f0b183b8bdf075ba30fe5f692b0d15853a1a17c4c33f766a1e2ee8a57399
  Stored in directory: /tmp/pip-ephem-wheel-cache-6o4j4yqo/wheels/9d/18/94/4155a499aea4ffd22b73a1983d594d4f4118e107068a30142f
  Created wheel for networkx: filename=networkx-2.3-py2.py3-none-any.whl size=1555990 sha256=a8f3afaf25f68a19883ce3979ae1d0207171e9ba387d20f945

In [5]:
import pandas as pd 
df=pd.read_csv('/content/drive/My Drive/ar_reviews_100k.tsv',sep='\t')
df.head()

,label,text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...


In [6]:
df['label'].value_counts()

Negative    33333
Positive    33333
Mixed       33333
Name: label, dtype: int64

In [7]:
### remove Mixed    label
df=df[df['label']!='Mixed']

In [8]:
### shuffle data
df=df.sample(frac=1).reset_index(drop=True)
df.head()

,label,text
0,Positive,يحتاج بعض التحسين . الراحةطاقم العملالخدمات. ا...
1,Positive,فيلم مقبول بختصار كون ان الفيلم ما زال لم يعرض...
2,Negative,فندق سيء . . تعامل الموظف المصري الذي وعدني بإ...
3,Negative,لم استوعب اي شيء الا مجموعه من ال لمات والطلاس...
4,Positive,جيد جدا. الجناح واسع جدا وكل شيء تقريبا متوفر ...


In [9]:
### hot encodding label (Positive --> 1 ,Negative -->0)
df['label']=df['label'].apply(lambda x: 1 if x=='Positive' else 0)
df.head()

,label,text
0,1,يحتاج بعض التحسين . الراحةطاقم العملالخدمات. ا...
1,1,فيلم مقبول بختصار كون ان الفيلم ما زال لم يعرض...
2,0,فندق سيء . . تعامل الموظف المصري الذي وعدني بإ...
3,0,لم استوعب اي شيء الا مجموعه من ال لمات والطلاس...
4,1,جيد جدا. الجناح واسع جدا وكل شيء تقريبا متوفر ...


In [10]:
from sklearn.model_selection import train_test_split
x=df['text']
y=df['label']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.15)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)


(56666,)
(56666,)
(10000,)
(10000,)


In [11]:
x_train.iloc[0]

'الشقق بشكل عام ممتازة . النظافة والراحة. لاشي'

In [12]:
y_train.iloc[0]

1

In [13]:
x_train = x_train.tolist()
y_train = y_train.to_numpy()
x_test = x_test.tolist()
y_test = y_test.to_numpy()

In [14]:
import ktrain
from ktrain import text
class_names=['Negative', 'Positive']
MODEL_NAME = 'aubmindlab/bert-base-arabertv01'

model=text.Transformer(MODEL_NAME,maxlen=300,class_names=class_names)
train=model.preprocess_train(x_train,y_train)
val=model.preprocess_test(x_test,y_test)
classifier=model.get_classifier()
learner=ktrain.get_learner(classifier,train_data=train, val_data=val,batch_size=6)


preprocessing train...
language: ar
train sequence lengths:
	mean : 54
	95percentile : 196
	99percentile : 459


Is Multi-Label? False
preprocessing test...
language: ar
test sequence lengths:
	mean : 56
	95percentile : 198
	99percentile : 481


In [15]:
learner.autofit(1e-6, 2)



begin training using triangular learning rate policy with max lr of 1e-06...
Epoch 1/2
9445/9445 [==============================] - 2550s 270ms/step - loss: 0.4002 - accuracy: 0.8078 - val_loss: 0.2720 - val_accuracy: 0.8856
Epoch 2/2
9445/9445 [==============================] - 2543s 269ms/step - loss: 0.2646 - accuracy: 0.8906 - val_loss: 0.2484 - val_accuracy: 0.8970


In [16]:
learner.validate(class_names=model.get_classes())

              precision    recall  f1-score   support

    Negative       0.91      0.88      0.90      5017
    Positive       0.88      0.92      0.90      4983

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



array([[4409,  608],
       [ 422, 4561]])

In [17]:
test_text = 'رائع جد هذا الفيلم ' # enter a review - ex. for printed above
predictor = ktrain.get_predictor(learner.model, preproc=model)
predictor.predict(test_text)

'Positive'

In [18]:
ktrain.get_predictor(learner.model, preproc=model).save('/content/drive/My Drive/arabic_bert')

# reload Predictor and extract model
#model = ktrain.load_predictor('/tmp/my_predictor').model


In [19]:

loaded_model = ktrain.load_predictor('/content/drive/My Drive/arabic_bert')

In [21]:
loaded_model.predict(test_text)

'Positive'